In [1]:
# # 매일 업데이트가 필요한 정보
# get_daily_OHLCV(start_date, end_date, stock_code)
# get_real_time_OHLCV(stock_code)
# get_index_csvs(target_business_year)
# crawl_sedaily_news(keyword, start_date, end_date)

# # 매주 업데이트가 필요한 정보
# crawl_ir_pdfs()

# # 매달 업데이트가 필요한 정보
# get_financial_info(stock_code, target_business_year)
# get_global_info(start_date, end_date)

from opendart import get_financial_info
from market_data import get_daily_OHLCV, get_real_time_OHLCV, get_index_csvs
from crawling import crawl_ir_pdfs, crawl_sedaily_news
from FRED import get_global_info

from util import get_quarterly_market_ticker_list

In [2]:
def update_all_raw_info(start_date:str, end_date:str):
    '''
    start_date, end_date 사이의

    ====================================================

    1. 주식 가격 데이터 (일별)
    2. 주요 산업 지표 (일별)
    3. 재무제표 (분기별)
    4. IR 자료 (업데이트 되는대로)
    5. 뉴스 헤드라인 (종목별, 일별)
    6. 뉴스 헤드라인 (경제 동향, 경제 주체 심리, 국제정치 동향, 일별)
    7. 글로벌 경제 지표 (미, EU, 영, 일, 중, 월별)
    
    ====================================================

    데이터를 모두 업데이트
    '''
    # 분기별로 종목리스트 나누어서 받아오기
    market_list = get_quarterly_market_ticker_list(start_date, end_date)
    quarter_keys = list(market_list.keys())
    print('=== 분기별 종목리스트 생성 완료 ===')

    is_index_csvs_updated = False

    # 분기별로 나눈 값에 대해 처리
    for i in range(len(quarter_keys)):
        print(f'=== {quarter_keys[i]}에 대한 정보 수집 시작 ({i+1}/{len(quarter_keys)}) ===')

        target_quarter = market_list[quarter_keys[i]]

        # 각 쿼터별 시작일, 종료일 및 티커 딕셔너리 ({티커}: {종목명}}) 를 각각 바인딩
        target_quarter_start_date = target_quarter['start_date']
        target_quarter_end_date = target_quarter['end_date']
        target_quarter_market_list = target_quarter['market_list']

        target_business_year = target_quarter_start_date[:4]

        # 각 분기에 해당하는 종목 리스트
        stock_code_list = list(target_quarter_market_list.keys())

        is_fin_info_updated = False
        ### === get_daily_OHLCV() 업데이트 ===
        ### === crawl_sedaily_news() 업데이트 (종목에 대한 크롤링 진행) === 
        ### === get_financial_info() 업데이트 ===
        for stock_code in stock_code_list:
            print(f'=== {quarter_keys[i]}의 {stock_code}에 대해 정보 수집 시작 ({stock_code_list.index(stock_code) + 1}/{len(stock_code_list)}) ===')

            print(f'=== {stock_code}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 가격데이터 저장 중... ===')
            get_daily_OHLCV(target_quarter_start_date, target_quarter_end_date, stock_code)
            print(f'=== {stock_code}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 가격데이터 저장 완료 ===')

            print(f'=== {stock_code}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 뉴스 헤드라인 데이터 저장 중... ===')
            crawl_sedaily_news(stock_code, target_quarter_start_date, target_quarter_end_date)
            print(f'=== {stock_code}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 뉴스 헤드라인 데이터 저장 완료 ===')

            if is_fin_info_updated == False:
                print(f'=== {stock_code}에 대한 {target_business_year} 재무 데이터 저장 중... ===')
                get_financial_info(stock_code, target_business_year)
                print(f'=== {stock_code}에 대한 {target_business_year} 재무 데이터 저장 완료 ===')

                is_fin_info_updated = True
            else:
                print(f'=== 이미 업데이트된 {stock_code}에 대한 {target_business_year} 재무 데이터입니다. ===')
                continue

        ### === get_index_csvs() 업데이트 ===
        # 인자로 연도를 받고, 한번만 실행해도 모든 연도에 대한 값이 들어오기 때문에 1번만 실행 (플래그를 활용하여 False일 때만 진행)
        if is_index_csvs_updated == False:
            print(f'=== {target_business_year} 산업군 지표 데이터 저장 중... ===')
            get_index_csvs(target_business_year)
            print(f'=== {target_business_year} 산업군 지표 데이터 저장 완료 ===')

            is_index_csvs_updated = True
        else:
            print(f'=== 이미 업데이트된 {target_business_year} 산업군 지표 데이터입니다. ===')
            continue

        ### === crawl_sedaily_news() 업데이트 (검색어에 대한 크롤링 진행) ===
        # 검색 시 활용할 키워드 모음.
        # 각각 경제동향, 경제 주체의 심리, 국제정치 동향에 대한 키워드
        search_keyword_list = [
            '금리 환율 물가 통화',
            '소비자 생산 투자',
            '전쟁 제재 갈등'
        ]
        for search_keyword in search_keyword_list:
            print(f'=== {search_keyword}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 뉴스 헤드라인 데이터 저장 중... ===')
            crawl_sedaily_news(search_keyword, target_quarter_start_date, target_quarter_end_date)
            print(f'=== {search_keyword}에 대한 {target_quarter_start_date}에서 {target_quarter_end_date}까지의 뉴스 헤드라인 데이터 저장 완료 ===')

    ### === crawl_ir_pdfs() 업데이트 ===
    # 한 번 실행 시 모든 정보를 다 가져오므로 분기별로 나누어 실행할 필요 없음
    print('=== IR 자료 저장 중... ===')
    crawl_ir_pdfs()
    print('=== IR 자료 저장 완료 ===')

    ### === get_global_info() 업데이트 ===
    # start_date와 end_date에 대해 한 번에 받아와도 되는 함수
    # FRED api로 가져오기 때문에 정보를 금방 가져오기도 하고, 분기별로 나누어 실행할 필요가 없음
    # 국내 주식장이 아니라 해외 경제 지표들에 대한 함수이기 때문
    print('=== 글로벌 경제 지표 저장 중... ===')
    get_global_info(start_date, end_date)
    print('=== 글로벌 경제 지표 저장 완료 ===')

    print(f'{start_date}에서 {end_date}까지의 정보를 모두 업데이트 하였습니다!')

In [3]:
# 전체 업데이트 코드
start_date = '20240630'
end_date = '20240701'

update_all_raw_info(start_date, end_date)

target_date_list [['20240701', '20240701']]
['20240701', '20240701']
key: 2024_3Q
=== 분기별 종목리스트 생성 완료 ===
=== 2024_3Q에 대한 정보 수집 시작 (1/1) ===
=== 2024_3Q의 095570에 대해 정보 수집 시작 (1/916) ===
=== 095570에 대한 20240701에서 20240701까지의 가격데이터 저장 중... ===
=== 095570에 대한 20240701에서 20240701까지의 가격데이터 저장 완료 ===
=== 095570에 대한 20240701에서 20240701까지의 뉴스 헤드라인 데이터 저장 중... ===
1/1 페이지 크롤링 중...
오류 발생: Message: no such element: Unable to locate element: {"method":"css selector","selector":".sub_news_list"}
  (Session info: chrome=128.0.6613.85); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000104f9b9a8 cxxbridge1$str$ptr + 1887096
1   chromedriver                        0x0000000104f93e10 cxxbridge1$str$ptr + 1855456
2   chromedriver                        0x0000000104b98be0 cxxbridge1$string$len + 89508
3   chromedriver                        0x00000001

KeyboardInterrupt: 

In [1]:
from market_data import get_daily_OHLCV, get_index_csvs
get_index_csvs('2019')

2019 코스피 CSV 파일 저장 1/46
2019 코스피 대형주 CSV 파일 저장 2/46
2019 코스피 중형주 CSV 파일 저장 3/46
2019 코스피 소형주 CSV 파일 저장 4/46
2019 음식료품 CSV 파일 저장 5/46
2019 섬유의복 CSV 파일 저장 6/46
2019 종이목재 CSV 파일 저장 7/46
2019 화학 CSV 파일 저장 8/46
2019 의약품 CSV 파일 저장 9/46
2019 비금속광물 CSV 파일 저장 10/46
2019 철강금속 CSV 파일 저장 11/46
2019 기계 CSV 파일 저장 12/46
2019 전기전자 CSV 파일 저장 13/46
2019 의료정밀 CSV 파일 저장 14/46
2019 운수장비 CSV 파일 저장 15/46
2019 유통업 CSV 파일 저장 16/46
2019 전기가스업 CSV 파일 저장 17/46
2019 건설업 CSV 파일 저장 18/46
2019 운수창고업 CSV 파일 저장 19/46
2019 통신업 CSV 파일 저장 20/46
2019 금융업 CSV 파일 저장 21/46
2019 증권 CSV 파일 저장 22/46
2019 보험 CSV 파일 저장 23/46
2019 서비스업 CSV 파일 저장 24/46
2019 제조업 CSV 파일 저장 25/46
2019 코스피 200 CSV 파일 저장 26/46
2019 코스피 100 CSV 파일 저장 27/46
2019 코스피 50 CSV 파일 저장 28/46
2019 코스피 200 커뮤니케이션서비스 CSV 파일 저장 29/46
2019 코스피 200 건설 CSV 파일 저장 30/46
2019 코스피 200 중공업 CSV 파일 저장 31/46
2019 코스피 200 철강/소재 CSV 파일 저장 32/46
2019 코스피 200 에너지/화학 CSV 파일 저장 33/46
2019 코스피 200 정보기술 CSV 파일 저장 34/46
2019 코스피 200 금융 CSV 파일 저장 35/46
2019 코스피 200 생활소비재 CSV 파일 저장 36/46


In [2]:
get_index_csvs('2020')
get_index_csvs('2021')
get_index_csvs('2022')
get_index_csvs('2023')

2020 코스피 CSV 파일 저장 1/46
2020 코스피 대형주 CSV 파일 저장 2/46
2020 코스피 중형주 CSV 파일 저장 3/46
2020 코스피 소형주 CSV 파일 저장 4/46
2020 음식료품 CSV 파일 저장 5/46
2020 섬유의복 CSV 파일 저장 6/46
2020 종이목재 CSV 파일 저장 7/46
2020 화학 CSV 파일 저장 8/46
2020 의약품 CSV 파일 저장 9/46
2020 비금속광물 CSV 파일 저장 10/46
2020 철강금속 CSV 파일 저장 11/46
2020 기계 CSV 파일 저장 12/46
2020 전기전자 CSV 파일 저장 13/46
2020 의료정밀 CSV 파일 저장 14/46
2020 운수장비 CSV 파일 저장 15/46
2020 유통업 CSV 파일 저장 16/46
2020 전기가스업 CSV 파일 저장 17/46
2020 건설업 CSV 파일 저장 18/46
2020 운수창고업 CSV 파일 저장 19/46
2020 통신업 CSV 파일 저장 20/46
2020 금융업 CSV 파일 저장 21/46
2020 증권 CSV 파일 저장 22/46
2020 보험 CSV 파일 저장 23/46
2020 서비스업 CSV 파일 저장 24/46
2020 제조업 CSV 파일 저장 25/46
2020 코스피 200 CSV 파일 저장 26/46
2020 코스피 100 CSV 파일 저장 27/46
2020 코스피 50 CSV 파일 저장 28/46
2020 코스피 200 커뮤니케이션서비스 CSV 파일 저장 29/46
2020 코스피 200 건설 CSV 파일 저장 30/46
2020 코스피 200 중공업 CSV 파일 저장 31/46
2020 코스피 200 철강/소재 CSV 파일 저장 32/46
2020 코스피 200 에너지/화학 CSV 파일 저장 33/46
2020 코스피 200 정보기술 CSV 파일 저장 34/46
2020 코스피 200 금융 CSV 파일 저장 35/46
2020 코스피 200 생활소비재 CSV 파일 저장 36/46


In [3]:
get_daily_OHLCV('20190101', '20240101', '005930')
get_daily_OHLCV('20190101', '20240101', '005380')

,시가,고가,저가,종가,거래량,등락률
날짜,,,,,,
2019-01-02,117500,118000,113500,114000,565493,-3.797468
2019-01-03,113000,118500,113000,116500,608591,2.192982
2019-01-04,116000,123000,116000,119500,1326978,2.575107
2019-01-07,121000,121500,119500,120500,428416,0.836820
2019-01-08,121500,123500,118500,119500,582117,-0.829876
...,...,...,...,...,...,...
2023-12-21,197500,202000,197100,198700,545031,-0.200904
2023-12-22,200000,200500,198500,199000,347866,0.150981
2023-12-26,200500,200500,197700,198800,409928,-0.100503
